In [1]:
"""Generative Adversarial Networks."""

from deepchem.models import TensorGraph
from deepchem.models.tensorgraph import layers
from collections import Sequence
import numpy as np
import tensorflow as tf
import time
import deepchem as dc
from deepchem.data.datasets import NumpyDataset # import NumpyDataset

import numpy as np
import tensorflow as tf
import collections

from deepchem.metrics import to_one_hot

from deepchem.models.tensorgraph.tensor_graph import TensorGraph, TFWrapper
from deepchem.models.tensorgraph.layers import Feature, Label, Weights, \
    WeightedError, Dense, Dropout, WeightDecay, Reshape, SoftMax, SoftMaxCrossEntropy, \
    L2Loss, ReduceSum, Concat, Stack

/Users/danieldemarchi/anaconda3/envs/deepchem/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/danieldemarchi/anaconda3/envs/deepchem/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
n_features = 1024
hiv_tasks, hiv_datasets, hiv_transformers = dc.molnet.load_hiv(featurizer='ECFP', split='random', reload=True)
hiv_train_dataset, hiv_valid_dataset, hiv_test_dataset = hiv_datasets

metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /var/folders/ph/d61mj9js3hvf9z7n92kk8y6c0000gn/T/HIV.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 28.452 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 1 took 35.083 s
Loading shard 3 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 2 took 26.859 s
Loading shard 

In [3]:
class HIVGAN(dc.models.WGAN):

  def get_noise_input_shape(self):
    return (None, 1024)

  def get_data_input_shapes(self):
    return [(None, 1024)]

  def create_generator(self, noise_input, conditional_inputs):
    dense1 = layers.Dense(1024, in_layers=noise_input, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense2 = layers.Dense(1024, in_layers=dense1, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    dense3 = layers.Dense(1024, in_layers=dense2, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    dense4 = layers.Dense(1024, in_layers=dense3, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    dense5 = layers.Dense(1024, in_layers=dense4, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    reshaped = layers.Reshape((None, 1024), in_layers=dense5)
    return [reshaped]

  def create_discriminator(self, data_inputs, conditional_inputs):
    dense1 = layers.Dense(1024, in_layers=data_inputs, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense2 = layers.Dense(1024, in_layers=dense1, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense3 = layers.Dense(1024, in_layers=dense2, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense4 = layers.Dense(1024, in_layers=dense3, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense5 = layers.Dense(1024, in_layers=dense4, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense6 = layers.Dense(1024, in_layers=dense5, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense7 = layers.Dense(1, in_layers=dense6, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    return dense7
#DISCRIMINATOR NEEDS TO BE BETTER!
gan = HIVGAN(learning_rate=0.0005)

In [4]:
def iterbatches(epochs):
  for i in range(epochs):
    print(gan.batch_size)
    for batch in hiv_train_dataset.iterbatches(batch_size=gan.batch_size):
      yield {gan.data_inputs[0]: batch[0]}

gan.fit_gan(iterbatches(33), generator_steps=1.5, checkpoint_interval=5000)

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
Ending global_step 5000: generator average loss nan, discriminator average loss nan
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
Ending global_step 10000: generator average loss nan, discriminator average loss nan
100
100
Ending global_step 10890: generator average loss nan, discriminator average loss nan
TIMING: model fitting took 5644.942 s


In [5]:
samples = gan.predict_gan_generator(batch_size=100000)
rsamples = np.rint(samples)

In [6]:
multitask_model = dc.models.MultitaskClassifier(
    1,
    n_features,
    layer_sizes=[1000],
    dropouts=[.25],
    learning_rate=0.001,
    batch_size=50)
multitask_model.fit(hiv_train_dataset)

7.878630254600454

In [7]:
samples = np.rint(samples)

In [8]:
resa = multitask_model.predict_on_batch(samples)[:,0]
print(np.count_nonzero(resa[:,1]))

100000


In [9]:
blah = np.rint(multitask_model.predict_on_batch(hiv_train_dataset.X)[:,0])
print(np.count_nonzero(blah[:,0]))
print(np.count_nonzero(blah[:,1]))

31119
1782


In [10]:
samples = gan.predict_gan_generator(batch_size=32901)
rsamples = np.rint(samples)

In [11]:
print(np.count_nonzero(rsamples))
print(np.count_nonzero(hiv_train_dataset.X))

33690624
1282501


In [12]:
class HIVGAN(dc.models.WGAN):

  def get_noise_input_shape(self):
    return (None, 1024)

  def get_data_input_shapes(self):
    return [(None, 1024)]

  def create_generator(self, noise_input, conditional_inputs):
    dense1 = layers.Dense(1024, in_layers=noise_input, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    dense2 = layers.Dense(1024, in_layers=dense1, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    dense3 = layers.Dense(1024, in_layers=dense2, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    dense4 = layers.Dense(1024, in_layers=dense3, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    dense5 = layers.Dense(1024, in_layers=dense4, activation_fn=tf.sigmoid, normalizer_fn=tf.layers.batch_normalization)
    reshaped = layers.Reshape((None, 1024), in_layers=dense5)
    return [reshaped]

  def create_discriminator(self, data_inputs, conditional_inputs):
    prev_layer = layers.Dense(1024, in_layers=data_inputs, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)
    for i in range(1, 5):
        layer = Dense(
          in_layers=prev_layer,
          out_channels=1024,
          activation_fn=tf.nn.relu,
          weights_initializer=TFWrapper(
              tf.truncated_normal_initializer, stddev=0.02),
          biases_initializer=TFWrapper(
              tf.constant_initializer, value=1))
        prev_layer = layer
        top_multitask_layer = prev_layer
        
    final = layers.Dense(1, in_layers=top_multitask_layer, activation_fn=tf.nn.relu, normalizer_fn=tf.layers.batch_normalization)

    return final
#DISCRIMINATOR NEEDS TO BE BETTER!
gan = HIVGAN(learning_rate=0.0005)

In [13]:
def iterbatches(epochs):
  for i in range(epochs):
    print(gan.batch_size)
    for batch in hiv_train_dataset.iterbatches(batch_size=gan.batch_size):
      yield {gan.data_inputs[0]: batch[0]}

gan.fit_gan(iterbatches(33), generator_steps=1.5, checkpoint_interval=200)

100
Ending global_step 200: generator average loss nan, discriminator average loss nan
100
Ending global_step 400: generator average loss nan, discriminator average loss nan
Ending global_step 600: generator average loss nan, discriminator average loss nan
100
Ending global_step 800: generator average loss nan, discriminator average loss nan
100
Ending global_step 1000: generator average loss nan, discriminator average loss nan
Ending global_step 1200: generator average loss nan, discriminator average loss nan
100
Ending global_step 1400: generator average loss nan, discriminator average loss nan
Ending global_step 1600: generator average loss nan, discriminator average loss nan
100
Ending global_step 1800: generator average loss nan, discriminator average loss nan
100
Ending global_step 2000: generator average loss nan, discriminator average loss nan
Ending global_step 2200: generator average loss nan, discriminator average loss nan
100
Ending global_step 2400: generator average loss 